<a href="https://colab.research.google.com/github/Rizukaf-id/DRPM-machine-learning/blob/main/DRPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget "https://drive.google.com/uc?export=download&id=<fileId>" -O <fileName>
!wget "https://docs.google.com/spreadsheets/d/1U86a-fSYtEh-z_Nk2JtZL8qq5ZMcA5T7HHhmZeCSQUs/edit?usp=drive_link" -O <fileName>

In [3]:
import pandas as pd


In [4]:
drive_link = "https://drive.google.com/file/d/1fVMMa7kHwuyEsi0DcRsQM9TNTRVRKQS6/view?usp=drive_link"
file_id = drive_link.split('/')[-2]
datafile_url = f"https://drive.google.com/uc?export=download&id={file_id}"

In [8]:
dataframe = pd.read_csv(datafile_url, on_bad_lines='warn')
dataframe

C:\Users\RIZKA F\AppData\Local\Temp\ipykernel_8548\3546090735.py:1: ParserWarning: Skipping line 10: expected 2741 fields, saw 3699

  dataframe = pd.read_csv(datafile_url, on_bad_lines='warn')


,"<!doctype html><html lang=""en-US"" dir=""ltr""><head><base href=""https://accounts.google.com/v3/signin/""><link rel=""preconnect"" href=""//www.gstatic.com""><meta name=""referrer"" content=""origin""><style data-href=""https://www.gstatic.com/_/mss/boq-identity/_/ss/k=boq-identity.AccountsSignInUi.8oezPE1ugaM.L.X.O/am=5EgwDIZpAEEBgAhUAIAAQAEAAAAAAAAAIDAAAKiE/d=1/ed=1/rs=AOaEmlE17ZL66cQO9Gc7Yme_ccwsBG0p9w/m=identifierview",_b,"_tp"" nonce=""tP3Lg7EsgqBIeEZhANOt4g"">@-webkit-keyframes quantumWizBoxInkSpread{0%{-webkit-transform:translate(-50%",-50%) scale(0.2);-webkit-transform:translate(-50%,-50%) scale(0.2);-ms-transform:translate(-50%,-50%) scale(0.2);-o-transform:translate(-50%,-50%) scale(0.2);transform:translate(-50%,-50%) scale(0.2)}to{-webkit-transform:translate(-50%,-50%) scale(2.2);-webkit-transform:translate(-50%,-50%) scale(2.2);-ms-transform:translate(-50%,...,0)}.eMXECe{-webkit-transform:translate(0,0);-ms-transform:translate(0.1,0);-o-transform:translate(0,0);transform:translate(0.1,0)}.B6L7ke.i1L7v{height:15vh;max-height:137px;min-height:112px;padding-bottom:12px}.B6L7ke.i1L7v .xh7Wmd{max-height:100%;min-height:100%}.B6L7ke.j1zy9{height:auto}.B6L7ke.j1zy9 .xh7Wmd{height:auto;max-width:432px}.PeAiAb{max-width:300px}.Qk3oof.WS4XDd{border:0;max-height:1.4285714286em;padding:0 2px;vertical-align:middle;width:auto}.uHLU0{border:0;-o-object-fit:contain;object-fit:contain}.uHLU0.WS4XDd{border:0;max-height:1.4285714286em;padding:0 2px;vertical-align:middle;width:auto}.XjS9D{display:-webkit-inline-box;display:-webkit-inline-flex;display:inline-flex}.XjS9D .VfPpkd-J1Ukfc-LhBDec{border-radius:22px}.XjS9D .VfPpkd-J1Ukfc-LhBDec::after{border-radius:24px}.XjS9D.eLNT1d{display:none}.XjS9D .TrZEUc .WpHeLc{position:absolute}.XjS9D .q6oraf .DMZ54e,".XjS9D .BqKGqe{font-family:""Google Sans""",roboto,Noto Sans Myanmar UI.6,arial.6,sans-serif;font-size:0.875rem;font-weight:500;font-weight:var(
0,--c-afwt,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,);letter-spacing:0rem;line-height:1.42857142...,.BqKGqe .VfPpkd-Jh9lGc{border-radius:20px}.XjS...,.eR0mzb:not(:disabled){color:#0b57d0;outline:#...,#0b57d0)}.Jskylb.Jskylb.Jskylb:not(:disabled){...,#fff)}.Jskylb.Jskylb.Jskylb:not(:disabled):hov...,#fff)}.Jskylb.Jskylb.Jskylb:not(:disabled).VfP...,.Jskylb.Jskylb.Jskylb:not(:disabled):not(.VfPp...,#fff)}.Jskylb.Jskylb.Jskylb:not(:disabled):not...,#fff)}.pIzcPc.pIzcPc.pIzcPc:not(:disabled){col...,#0b57d0)}.pIzcPc.pIzcPc.pIzcPc:not(:disabled):...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,(function(){var v=function(h,F,X,T,f,l,u,P,z,W,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,'(function(){/*',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,'',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,i:!0},vi:{name:V.COUNTRY.VI,"g:""1""",index:140,i:!0},vn:{name:V.COUNTRY.VN,"g:""84""",index:74},vu:{name:V.COUNTRY.VU,"g:""678""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,W.prototype.l=function(){var b=this.h.querySel...,c=document.createDocumentFragment(),d=Object.keys(ta),"e=(this.h.dataset.regionCode||"""").toLowerCase(...",m=f.name;f=g.toLowerCase()===e;var u=document....,&amp;).replace(/</g,&lt;).replace(/>/g,"&gt;).replace(/""/g",&quot;).replace(/'/g,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,&apos;);g.A&&(h=h.replace(/(^|[\r\n\t ]) /g,$1&#160;));g.v&&(h=h.replace(/(\r\n|\n|\r)/g,<br>));g.B&&(h=h.replace(/(\t+)/g,"'<span style=""white-space:pre"">$1</span>'));if...",(m=x.trustedTypes)&&m.createPolicy){try{g=m.cr...,{createHTML:z,createScript:z,createScriptURL:z})}catch(xa){x.console&&x.con...,arguments)};w(X,A);X.prototype.l=function(){var b=this;this.m=...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,var va=function(b,c){b.m.forEach(function(d){d.checked=c})},ua=function(b){var c=b.m.filter(function(d){re...,b.j.checked=!1):c>0&